# Modèle 1 : Finalisation dernière minute

- On ajoute des poids pour bénéficier "Définition" \*1.3 et réduire "Titre" \*0.7

## Imports

In [2]:
!pip install pandas spacy bs4 sentence_transformers numpy
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 4.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [3]:
import pandas as pd
import re
import unicodedata
import spacy
from bs4 import BeautifulSoup
from spacy.lang.fr.stop_words import STOP_WORDS
import pickle
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.quantization import quantize_embeddings
import numpy as np
import copy

/home/patrice/Documents/Etudes/CY-Tech/Ing2_Sem2_2023-2024/UE7_Projet_DataBattle_IA-Pau/ia-pau-data-battle-mars-2024/backend/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Nouvelles fonctions

### On traite les données

In [4]:
def load_and_merge_data(csv_file='../data/solutions2.csv'):
    dictionnaire_solution = {1: "Titre", 2: "Description", 5: "Application", 6: "Bilan énergétique", 21: "Titre technologie", 22: "Description technologie"}

    # Charger le fichier CSV en spécifiant le séparateur '|'
    df = pd.read_csv(csv_file, sep='|', header=None)

    # Initialiser une liste pour stocker les données de chaque solution
    solutions_data = []

    # Parcourir chaque ligne du DataFrame
    for index, row in df.iterrows():
        id_sol = row[0]
        section = row[1]
        texte = row[2]

        # Vérifier si la section correspond à une clé dans le dictionnaire de solutions
        if section in dictionnaire_solution:
            # Récupérer le nom de la section
            section_name = dictionnaire_solution[section]

            # Chercher si la solution existe déjà dans la liste
            solution_exists = False
            for solution in solutions_data:
                if solution[0] == id_sol:
                    solution_exists = True
                    solution[1][section_name] = texte
                    break

            # Si la solution n'existe pas encore, la créer
            if not solution_exists:
                new_solution = [id_sol, {section_name: texte}]
                solutions_data.append(new_solution)

    return solutions_data

In [35]:
# Appel de la fonction pour obtenir les données
df_solutions = load_and_merge_data()
print(df_solutions[0])

[2, {'Titre technologie': 'Centrale froid', 'Titre': 'Installation frigorifique négative de type cascade utilisant du CO2', 'Description': 'Mise en place dune installation frigorifique négative de type cascade utilisant du CO2 comme fluide frigorigène.', 'Application': 'Pour être éligible à CEE, la mise en place doit être effectuée par un professionnel et appliquée dans des locaux de commerce de distribution alimentaire de surface de vente inférieure à 5000 m². Comparé aux autres fluides frigorigènes, le CO2 est un fluide'}]


In [6]:
# Charger le modèle de langue SpaCy
nlp = spacy.load("fr_core_news_sm")

def clean_df_solutions(df_solutions):
    cleaned_data = []
    for item in df_solutions:
        index = item[0]
        solution = item[1]
        cleaned_solution = {}

        for key, value in solution.items():
            if (key == 'Titre' or key == 'Titre technologie') :
                # Pour les titres, ne pas enlever les chiffres
                cleaned_solution[key] = clean_text(str(value), remove_numbers=False)
            else:
                # Pour les autres champs, enlever les chiffres
                cleaned_solution[key] = clean_text(str(value), remove_numbers=True)

        cleaned_data.append([index, cleaned_solution])

    return cleaned_data

def clean_text(text, remove_numbers=True):
    # Nettoyer HTML Tags
    text = BeautifulSoup(text, 'html.parser').get_text()

    # Remplacer "&nbsp;." par rien
    text = re.sub(r'&nbsp;\.', '', text)

    # Supprimer les "l'"
    text = re.sub(r"\bl'", '', text)

    # Accents
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")

    if remove_numbers:
        # Retirer les numéros
        text = re.sub(r'\b\d+\b', '', text)

    # Supprimer les caractères seuls
    text = re.sub(r'\b\w\b', '', text)

    # Tokenization, Lemmatization, Removing Stopwords, Lowercase
    doc = nlp(text)
    cleaned_sentences = []
    for sentence in doc.sents:
        tokens = [token.lemma_.lower() for token in sentence if not token.is_stop and not token.is_punct and not token.is_space]
        clean_sentence = ' '.join(tokens)
        if clean_sentence:
            cleaned_sentences.append(clean_sentence)  # Ajouter un point à la fin de la phrase propre

    # Joining the cleaned sentences back into a single string
    cleaned_text = ' '.join(cleaned_sentences)

    return cleaned_text

In [36]:
df_solutions_clean = clean_df_solutions(df_solutions)

/tmp/ipykernel_28664/1373267169.py:25: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


In [37]:
print(df_solutions_clean[4])

[6, {'Titre technologie': 'condenseur', 'Titre': 'condenseur frigorifique haute efficacite', 'Description': 'mise place condenseur haute efficacite installation frigorifique neuf existant certification eurovent exemple garer qualite condenseur', 'Application': 'condenseur evaporatif efficace utiliser chaleur latent evacuer chaleur plupart systeme refrigerant utiliser condenseur refroidi lair levacuation chaleur condenseur evaporation utiliser filtre humidifie refroidir lair ambier entree condenseur accroître capacite devacuation chaleur', 'Bilan énergétique': 'condenseur haute efficacite energetiqu echangeur presenter faible ecart temperature lecart temperature fluide frigorigene pression condensation medium refroidissement eau air entree condenseur abaisser dabaisser consommation groupe frigorifique efficacite production frigorifique cop coefficient performance dun machine correspondre rapport quantite chaleur recupere condenseur quantite denergie electriqu total absorbee linstallatio

Pour vérifier qu'on garde les numéros

In [9]:
# def get_solution_by_id(df_solutions_clean, solution_id):
#     for item in df_solutions_clean:
#         if item[0] == solution_id:
#             return item[1]
#     return None

# # Utilisation de la fonction pour obtenir le dictionnaire de l'ID 22
# solution_id_choisi = 1692
# solution_choisie = get_solution_by_id(df_solutions_clean, solution_id_choisi)

# # Afficher le dictionnaire de la solution choisie
# print(solution_choisie)

### On charge le modèle 

In [34]:
# Charger le modèle
model = SentenceTransformer("dangvantuan/sentence-camembert-large")

No sentence-transformers model found with name dangvantuan/sentence-camembert-large. Creating a new one with MEAN pooling.


## Sauvegarde des embeddings

On va sauvegarder les embeddings en gardant le nom des champs.

In [40]:
def encoder_embeddings(data, model, output_file):
    # Faire une copie de data
    data = copy.deepcopy(data)
    

    # Fonction pour encoder chaque texte
    def encoder_texte(texte):
        return model.encode(texte)
    

    # Pour chaque entrée dans les données, encoder tous les champs texte
    for entry in data:
        # print("DEBUG : entry = ", entry)
        for champ, valeur in entry[1].items():
            # print("DEBUG : champ =", champ, ", valeur = ", valeur)
            if isinstance(valeur, str):  # S'assurer que la valeur est une chaîne de caractères
                # Calculer l'embedding
                embedding = encoder_texte(valeur)
                # Remplacer le texte par l'embedding
                entry[1][champ] = embedding.tolist()
    # Sauvegarder les embeddings dans un fichier
    with open(output_file, 'wb') as file:
        pickle.dump(data, file)
    return data

In [12]:
def calculate_average_embedding(text, model):
    # Diviser le texte en phrases
    sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]
    sentence_embeddings = model.encode(sentences)
    # Prendre la moyenne des embeddings des phrases
    if len(sentence_embeddings) > 0:
        average_embedding = np.mean(sentence_embeddings, axis=0)
    else:
        average_embedding = np.zeros(model.get_sentence_embedding_dimension())
    return average_embedding

def encoder_embeddings_moyenne_sentences(data, model, output_file):
    # Faire une copie de data
    data = copy.deepcopy(data)
    # Pour chaque entrée dans les données, encoder tous les champs texte
    for entry in data:
        # print("DEBUG : entry = ", entry)
        for champ, valeur in entry[1].items():
            # print("DEBUG : champ =", champ, ", valeur = ", valeur)
            if isinstance(valeur, str):  # S'assurer que la valeur est une chaîne de caractères
                # Calculer l'embedding en faisant la moyenne de l'embedding de chaque phrase.
                embedding = calculate_average_embedding(valeur, model)
                # Remplacer le texte par l'embedding
                entry[1][champ] = embedding.tolist()
    # Sauvegarder les embeddings dans un fichier
    with open(output_file, 'wb') as file:
        pickle.dump(data, file)

    return data

Commençons par calculer un embedding sur deux solutions :

In [42]:
path_embedding_file = "embeddings/FR_camembert_large_avec_moyenne_phrases.pkl"
solutions_embeddings = encoder_embeddings_moyenne_sentences(df_solutions_clean, model, path_embedding_file)

In [14]:
# path_embedding_file = "embeddings/model_final_moyenne_sentences_magb.pkl"
# solutions_embeddings = encoder_embeddings_moyenne_sentences(df_solutions_clean, model, path_embedding_file)

In [15]:
# # Pour voir notre embedding sur notre première solution.
# for champ, valeur in solutions_embeddings[0][1].items():
#     print(champ," : ", valeur)

In [16]:
# print(solutions_embeddings[0])

## Inférence

In [17]:
# Fonction appelé par notre utilisateur
def model_find_solution(description, secteur, model, embeddings_file_path, seuil_similarite, min_sol, weights) :
    # !!!!!!!!!!!! ICI ON A CHOISI DE NE PAS INCLURE LE SECTEUR DANS NOTRE MODELE, COMMENTER LA LIGNE CI-DESOSUS POUR CHANGER CELA --------------
    secteur = ""
    # -------------------------

    # On commence par concaténer notre secteur et notre description.
    requete = secteur + ". " + description

    # Ensuite on applique notre pré-processing
    clean_requete = clean_text(requete, remove_numbers=False)
    clean_requete_vecteur =  model.encode(clean_requete)

    # On va lire notre fichier d'embeddings 
    with open(embeddings_file_path, "rb") as fIn:
        solutions_embeddings = pickle.load(fIn)


    solutions_similarities = []
    # Maintenant pour chaque solution on va garder notre meilleur similarité cosinus avec notre requete. De plus nous ajouton un poids à chaque champs de notre solution,
    # Description * 1.3, Titre * 0.7, et 1 pour tous les autres.
    # On se retrouve donc avec une liste [[id_sol, max_similarité], ...]

    # Pour chaque entrée dans les données, encoder tous les champs texte
    for entry in solutions_embeddings:
        max_similarity = 0
        id_solution = entry[0]
        for champ, valeur_vecteur in entry[1].items():
            # Coeficient multiplicateur 
            weight = weights[champ]
            # On calcul la similarité
            similarity = util.pytorch_cos_sim(valeur_vecteur, clean_requete_vecteur)*weight
            # On met à jour le max de similarité
            if similarity > max_similarity :
                max_similarity = similarity
        solution_similarity = [id_solution, max_similarity]
        solutions_similarities.append(solution_similarity)
    #print("DEBUG : ",solutions_similarities)
    
    
    # On trie notre liste de solution par ordre croissant
    solutions_similarities.sort(key=lambda x: x[1], reverse=True)

    # On sélectionne les 5 meilleures solutions
    top_5_solutions = [sol[0] for sol in solutions_similarities[:min_sol]]

    # On applique le seuil de similarité aux solutions restantes
    remaining_solutions = [sol[0] for sol in solutions_similarities[min_sol:] if sol[1] > seuil_similarite]

    # On concatène les deux listes de solutions
    final_solutions = top_5_solutions + remaining_solutions

    # On retourne que les solutions et non la similarité
    return final_solutions

In [ ]:
weights = {"Titre":1, "Description":1, "Application":1, "Bilan énergétique":1, "Titre technologie":1, "Description technologie":1}

In [19]:
# path_embedding_file = "embeddings/model_final_magb_avant_ajout_points.pkl"
# model_find_solution("C'est quoi la HP flottante ?", "", model, path_embedding_file, 0.7, 5, weights)

In [20]:
# path_embedding_file = "embeddings/model_final_magb.pkl"
# model_find_solution("C'est quoi la HP flottante ?", "", model, path_embedding_file, 0.7, 5, weights)

In [21]:
# path_embedding_file = "embeddings/model_final_moyenne_sentences_magb.pkl"
# model_find_solution("C'est quoi la HP flottante ?", "", model, path_embedding_file, 0.7, 5, weights)

In [22]:
path_embedding_file = "embeddings/model_final_magb_sans_somme_sentences.pkl"
model_find_solution("C'est quoi la HP flottante ?", "", model, path_embedding_file, 0.7, 5, weights)

[724, 835, 1533, 1608, 1595]

In [44]:
path_embedding_file = "embeddings/FR_camembert_large_avec_moyenne_phrases.pkl"
model_find_solution("C'est quoi la HP flottante ?", "", model, path_embedding_file, 0.7, 5, weights)

[724, 1604, 1602, 1603, 1605]

## Tester sur dataset Kerdos

In [23]:
# Lire le fichier CSV
df_testset = pd.read_csv("data/dataset_test_Kerdos.csv")
#df_testset = pd.read_csv("data/patrice_test_set.csv")


def test_accuracy(path_embedding_file, dataset=df_testset, top_n=1) :
    accuracy = 0
    for i in range(1,len(dataset)):
        predictions = model_find_solution(dataset['Description'][i], "", model, path_embedding_file, 0.8, 5, weights)
        # print("---------------------------------------")
        # print("Valeur attendu : ", dataset["id_solution"][i])
        # print(predictions)
        if (dataset["id_solution"][i] in predictions[:top_n]):
            accuracy += 1/len(dataset)
        # else :
        #     print("mal prédit : ", dataset["id_solution"][i])
    return accuracy

In [24]:
# path_embedding_file = "embeddings/model_final_magb_avant_ajout_points.pkl"
# test_accuracy(path_embedding_file)

In [25]:
# path_embedding_file = "embeddings/model_final_magb.pkl"
# test_accuracy(path_embedding_file)

In [26]:
# path_embedding_file = "embeddings/model_final_moyenne_sentences_magb.pkl"
# test_accuracy(path_embedding_file)

In [27]:
path_embedding_file = "embeddings/model_final_magb_sans_somme_sentences.pkl"
test_accuracy(path_embedding_file)

0.25

In [45]:
path_embedding_file = "embeddings/FR_camembert_large_avec_moyenne_phrases.pkl"
test_accuracy(path_embedding_file)

0.1875

# FONCTIONS POUR OPTIMISER ACCURACY

## Répétition des fonctions à run exclusivement pour ces tests

### Dataset et weights

In [46]:
# Lire le fichier CSV
df_testset = pd.read_csv("data/dataset_test_Kerdos.csv")
#df_testset = pd.read_csv("data/patrice_test_set.csv")

weights = {"Titre":1, "Description":1, "Application":1, "Bilan énergétique":1, "Titre technologie":1, "Description technologie":1}

## Algo d'optimisation

In [29]:
# # Importations nécessaires
# import itertools

# # Définition des poids à tester
# weights_to_try = {
#     "Titre": [0.5, 1],
#     "Description": [0.5, 1],
#     "Application": [0.5, 1],
#     "Bilan énergétique": [0.5, 1],
#     "Titre technologie": [0.5, 1],
#     "Description technologie": [0.5, 1],
# }

# best_accuracy = 0.0
# best_weights = None

# # Parcourir toutes les combinaisons de poids
# for weights_combination in itertools.product(*weights_to_try.values()):
#     # Créer un dictionnaire de poids à partir de la combinaison
#     weights = dict(zip(weights_to_try.keys(), weights_combination))
    
#     # Calculer l'accuracy pour cette combinaison de poids
#     accuracy = test_accuracy(path_embedding_file, df_testset, top_n=3)
    
#     # Vérifier si c'est la meilleure accuracy jusqu'à présent
#     if accuracy > best_accuracy:
#         best_accuracy = accuracy
#         print("Best accuracy : ", best_accuracy)
#         best_weights = weights.copy()
#         print("Best weight : ", best_weights)

# print("Meilleure accuracy trouvée:", best_accuracy)
# print("Meilleurs poids associés:")
# print(best_weights)

In [47]:
import random
import json

# Définition des poids à tester
weights_to_try = {
    "Titre": [0.5,0.7,1.3,1.5,1.7,1.9,2,2],
    "Description": [0.5,0.7,1.3,1.5,1.7,1.9,2,2],
    "Application": [0.5,0.7,1.3,1.5,1.7,1.9,2,2],
    "Bilan énergétique": [0.5,0.7,1.3,1.5,1.7,1.9,2,2],
    "Titre technologie": [0.5,0.7,1.3,1.5,1.7,1.9,2,2],
    "Description technologie": [0.5,0.7,1.3,1.5,1.7,1.9,2,2],
}

best_accuracy_top_1 = 0.0
best_accuracy_top_3 = 0.0
best_weights = None
num_iterations = 1500  # Nombre d'itérations pour la recherche aléatoire

def save_best_model(best_accuracy, best_weights, path):
    with open(path, "w") as f:
        model_data = {
            "best_accuracy": best_accuracy,
            "best_weights": best_weights
        }
        json.dump(model_data, f, indent=4)

for _ in range(num_iterations):
    # Générer une combinaison aléatoire de poids
    weights = {key: random.choice(values) for key, values in weights_to_try.items()}
    
    # Calculer l'accuracy pour cette combinaison de poids
    accuracy_top_1 = test_accuracy(path_embedding_file, df_testset, top_n=1)
    accuracy_top_3 = test_accuracy(path_embedding_file, df_testset, top_n=3)

    # Vérifier si c'est la meilleure accuracy jusqu'à présent
    if accuracy_top_1 > best_accuracy_top_1:
        best_accuracy_top_1 = accuracy_top_1
        print("Best accuracy : ", best_accuracy_top_1)
        best_weights = weights.copy()
        print("Best weight : ", best_weights)
        # Sauvegarder les meilleurs poids et l'accuracy dans un fichier
        save_best_model(best_accuracy_top_1, best_weights, "best_weights_camembert_top_1.json")

    # Vérifier si c'est la meilleure accuracy jusqu'à présent
    if accuracy_top_3 > best_accuracy_top_3:
        best_accuracy_top_3 = accuracy_top_3
        print("Best accuracy : ", best_accuracy_top_3)
        best_weights = weights.copy()
        print("Best weight : ", best_weights)
        # Sauvegarder les meilleurs poids et l'accuracy dans un fichier
        save_best_model(best_accuracy_top_3, best_weights, "best_weights_camembert_top_3.json")
    


Best accuracy :  0.375
Best weight :  {'Titre': 2, 'Description': 2, 'Application': 2, 'Bilan énergétique': 2, 'Titre technologie': 1.5, 'Description technologie': 0.7}
Best accuracy :  0.625
Best weight :  {'Titre': 2, 'Description': 2, 'Application': 2, 'Bilan énergétique': 2, 'Titre technologie': 1.5, 'Description technologie': 0.7}
Best accuracy :  0.6875
Best weight :  {'Titre': 2, 'Description': 1.9, 'Application': 1.9, 'Bilan énergétique': 1.7, 'Titre technologie': 0.7, 'Description technologie': 1.5}
Best accuracy :  0.5
Best weight :  {'Titre': 1.7, 'Description': 2, 'Application': 0.7, 'Bilan énergétique': 1.7, 'Titre technologie': 0.7, 'Description technologie': 2}
Best accuracy :  0.75
Best weight :  {'Titre': 1.7, 'Description': 2, 'Application': 1.5, 'Bilan énergétique': 0.5, 'Titre technologie': 1.9, 'Description technologie': 1.5}


KeyboardInterrupt: 